# Some predictions and plots

In [8]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import sys
from importlib import reload

# Set the working directory
os.chdir("D:/projects/IPTLC_BBMs/")
print(f"Working directory: {os.getcwd()}")

# Import local modules
import src.creator as creator
import src.models.OPF as opf_models

reload(creator)
reload(opf_models)

Working directory: D:\projects\IPTLC_BBMs


<module 'src.models.OPF' from 'D:\\projects\\IPTLC_BBMs\\src\\models\\OPF.py'>

In [9]:
# Check GPU availability
device = creator.get_device()
print(device)

cuda:0


In [10]:
# Specify the paths
datasets_folder = Path("data/OPF/2023-12-06_18-00-46")

# Let's open single simulation
single_input_normalized = np.load(datasets_folder / Path("OPF-NI1-O3/single_input.npy"))
single_input_raw = np.load(datasets_folder / Path("OPF-I1-O3/single_input.npy"))
single_output = np.load(datasets_folder / Path("OPF-I1-O3/single_output.npy"))

# Convert to torch tensors
single_input_normalized = torch.from_numpy(single_input_normalized).float().to(device)
single_input_raw = torch.from_numpy(single_input_raw).float().to(device)
single_output = torch.from_numpy(single_output).float().to(device)

In [11]:
# Size of the input and output
input_size = single_input_normalized.shape[1]
output_size = single_output.shape[1]

print(f"Input size: {input_size}")
print(f"Output size: {output_size}")

Input size: 53
Output size: 9


In [12]:
# Load the single example dataframes to extract the input/output names from the columns
input_df = pd.read_csv(datasets_folder / Path("OPF-I1-O3/single_input.csv"), index_col=0)
output_df = pd.read_csv(datasets_folder / Path("OPF-I1-O3/single_output.csv"), index_col=0)

# Get the names of the inputs and outputs
input_names = input_df.columns.values.tolist()
output_names = output_df.columns.values.tolist()

In [14]:
# Load the models trained with the normalized dataset
OPF_BBM_1_normalized = opf_models.Net(input_size, output_size)
OPF_BBM_1_normalized.load_state_dict(torch.load("models/BBM1-2layers_OPF-NI1-O3_20240201-110613.pt"))
OPF_BBM_1_normalized.to(device)
OPF_BBM_1_normalized.eval()

OPF_BBM_2_normalized = opf_models.DeepNN(input_size, output_size)
OPF_BBM_2_normalized.load_state_dict(torch.load("models/BBM2-deep_OPF-NI1-O3_20240201-112030_best.pt"))
OPF_BBM_2_normalized.to(device)
OPF_BBM_2_normalized.eval()

# Load the models trained with the raw dataset
OPF_BBM_1_raw = opf_models.Net(input_size, output_size)
OPF_BBM_1_raw.load_state_dict(torch.load("models/BBM1-2layers_OPF-I1-O3_20240201-111317.pt"))
OPF_BBM_1_raw.to(device)
OPF_BBM_1_raw.eval()


BBM1_SimpleNet(
  (fc1): Linear(in_features=53, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=9, bias=True)
  (dropout): Dropout(p=0.15, inplace=False)
)

In [25]:
## Predictions using BBM1 with raw and normalized data
# Predictions using the normalized dataset and BBM1
prediction_normalized_1 = OPF_BBM_1_normalized(single_input_normalized)

# Predictions using the raw dataset and BBM1
prediction_raw_1 = OPF_BBM_1_raw(single_input_raw)
# Plot the predictions versus the ground truth
# Obtain the size of the output. THen, create an equal number of subplots and plot predictions versus ground truth in each subplot
print(f"Output size: {output_size}")

# Calculate a nice number of rows and columns for the subplots
figsize = (20, 10)
rows = int(np.sqrt(output_size))
cols = int(np.ceil(output_size / rows))

# Create the figure
fig, axs = plt.subplots(rows, cols, figsize=figsize, constrained_layout=True)

try:
    y_limits
except NameError:
    y_limits = None

# Plot the predictions versus the ground truth
for i in range(rows * cols):
    # Delete the axis if there is no data to plot
    if i >= output_size:
        axs[i // cols, i % cols].axis("off")
        continue

    # Calculate the row and column of the subplot
    row = i // cols
    col = i % cols

    # Plot the prediction versus the ground truth
    axs[row, col].plot(single_output[:, i].cpu().detach().numpy(), label="Ground truth", color="black")
    axs[row, col].plot(prediction_raw_1[:, i].cpu().detach().numpy(), label="Prediction (raw input)", color="black",
                       linestyle="--")
    axs[row, col].plot(prediction_normalized_1[:, i].cpu().detach().numpy(), label="Prediction (normalized input)",
                       color="black", linestyle=":")
    axs[row, col].set_title(output_names[i])

    # Set the x limit
    axs[row, col].set_xlim([0, 96])

    # Set the y limit
    if y_limits is not None:
        # get the current y limits
        this_y_bot, this_y_up = axs[row, col].get_ylim()
        gen_y_bot, gen_y_up = y_limits[i]

        # check both limits: if bottom limit is lower than the current one, update it and viceversa
        if this_y_bot < gen_y_bot:
            y_limits[i][0] = this_y_bot
        if this_y_up > gen_y_up:
            y_limits[i][1] = this_y_up

        # set the limits
        axs[row, col].set_ylim(y_limits[i])

# Add a legend at the top
_ = fig.legend(["Ground truth", "Prediction (raw input)", "Prediction (normalized input)"], loc="upper center", ncol=3,bbox_to_anchor=(0.5, 1.05))

-0.16389999999999993

In [27]:
# Predictions using the normalized dataset and BBM1
prediction_normalized_2 = OPF_BBM_2_normalized(single_input_normalized)

301 µs ± 87.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [28]:
# Plot the predictions versus the ground truth
# Obtain the size of the output. THen, create an equal number of subplots and plot predictions versus ground truth in each subplot
output_size = single_output.shape[1]
print(f"Output size: {output_size}")

# Calculate a nice number of rows and columns for the subplots
figsize = (20, 10)
rows = int(np.sqrt(output_size))
cols = int(np.ceil(output_size / rows))

# Create the figure
fig, axs = plt.subplots(rows, cols, figsize=figsize, constrained_layout=True)

# Plot the predictions versus the ground truth
for i in range(rows * cols):
    # Delete the axis if there is no data to plot
    if i >= output_size:
        axs[i // cols, i % cols].axis("off")
        continue

    # Calculate the row and column of the subplot
    row = i // cols
    col = i % cols

    # Plot the prediction versus the ground truth
    axs[row, col].plot(single_output[:, i].cpu().detach().numpy(), label="Ground truth", color="black")
    axs[row, col].plot(prediction_normalized_1[:, i].cpu().detach().numpy(), label="Prediction (BBM1)", color="black", linestyle=":")
    axs[row, col].plot(prediction_normalized_2[:, i].cpu().detach().numpy(), label="Prediction (BBM2)", color="red", linestyle="-.")
    axs[row, col].set_title(output_names[i])

    # Set the x limit
    axs[row, col].set_xlim([0, 96])

    # Set the y limit
    if y_limits is not None:
        # get the current y limits
        this_y_bot, this_y_up = axs[row, col].get_ylim()
        gen_y_bot, gen_y_up = y_limits[i]

        # check both limits: if bottom limit is lower than the current one, update it and viceversa
        if this_y_bot < gen_y_bot:
            y_limits[i][0] = this_y_bot
        if this_y_up > gen_y_up:
            y_limits[i][1] = this_y_up

        # set the limits
        axs[row, col].set_ylim(y_limits[i])

# Add a legend at the top
_ = fig.legend(["Ground truth", "Prediction (BBM1)", "Prediction (BBM2)"], loc="upper center", ncol=3, bbox_to_anchor=(0.5, 1.05))

1.2499999999999999e-05